# Part 1

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

In [5]:
lines = None
with open('q1-test.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
observs = [None]*len(temp)
for idx, i in enumerate(temp):
    observs[idx] = [ord(j) - ord('a') for j in i]
for i in range(len(observs)):
    observs[i] = np.array(observs[i])

In [37]:
def normalize(arr):
    for i in range(len(arr)):
        arr[i] = arr[i] / np.sum(arr[i])
    return arr

def forward(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    
    alpha = np.zeros((N, T))
    alpha[:, 0] = pi * B[:, obs[0]]
    alpha[:, 0] /= np.sum(alpha[:, 0])
    for j in range(1, T):
        for i in range(N):
            alpha[i, j] = B[i, obs[j]] * (alpha[:, j - 1] @ A[:, i])
        alpha[:, j] /= np.sum(alpha[:, j])
    
    return alpha

def backward(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    
    beta = np.zeros((N, T))
    for i in range(N):
        beta[i, T - 1] = 1
    
    for i in range(1, T):
        j = T - i - 1
        beta[:, j] = A @ (beta[:, j + 1] * B[:, obs[j + 1]])
    return beta
    
    
def new_matrices(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    alpha = forward(A, B, pi, obs)
    beta = backward(A, B, pi, obs)
    
    gamma = np.zeros((N, T))
    
    for j in range(1, T):
        for i in range(N):
            gamma[i, j] = (alpha[i, j] * beta[i, j])/(alpha[:, j] @ beta[:, j])
            
    psi = np.zeros((N, N, T - 1))
    for i in range(N):
        for j in range(N):
            for k in range(T - 1):
                psi[i, j, k] = (alpha[i, k] * A[i, j] * beta[j, k + 1] * B[j, obs[k + 1]])/(alpha[:, k] @ (A @ (beta[:, k + 1] * B[:, obs[k + 1]])))
    k = B.shape[1]
    new_pi = gamma[:, 0]
    new_A = sum([psi[:, :, i] for i in range(T - 1)]) / sum([gamma[:, i] for i in range(T - 1)])
    new_B = np.zeros((N, k))
    for i in range(N):
        for j in range(k):
            new_B[i, j] = np.sum([i for idx, i in enumerate(gamma[i]) if obs[idx] == j])/np.sum(gamma[i])
    return normalize(new_A), normalize(new_B), new_pi / (np.sum(new_pi) + 1e-6)

In [42]:


A = normalize(np.random.uniform(size=(5, 5)))
B = normalize(np.random.uniform(size=(5, 7)))
pi = np.random.uniform(size=(5))
pi /= np.sum(pi)



new_matrices(A, B, pi, observs[0])

def baum_welch_multiple(A, B, pi, observs, n=50):
    
    for k in range(n):
        new_A = np.zeros(shape=A.shape)
        new_B = np.zeros(shape=B.shape)
        new_pi = np.zeros(shape=pi.shape)
        for i in observs:
            temp1, temp2, temp3 = new_matrices(A, B, pi, i)
            new_A += temp1
            new_B += temp2
            new_pi += temp3

        new_A /= len(observs)
        new_B /= len(observs)
        new_pi /= len(observs)

        A = new_A
        B = new_B
        pi = new_pi
    return A, B, pi
baum_welch_multiple(A, B, pi, observs)
        


<ipython-input-37-8cc6ac43fb8f>:13: RuntimeWarning: invalid value encountered in true_divide
  alpha[:, 0] /= np.sum(alpha[:, 0])


(array([[nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan]]),
 array([[nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan],
        [nan, nan, nan, nan, nan, nan, nan]]),
 array([0., 0., 0., 0., 0.]))

In [43]:
def forward_temp(A, B, pi, obs):
    T = len(obs)
    N = A.shape[0]
    k = B.shape[1]
    
    alpha = np.zeros((N, T))
    alpha[:, 0] = pi * B[:, obs[0]]
    alpha[:, 0] /= np.sum(alpha[:, 0])
    for j in range(1, T):
        for i in range(N):
            alpha[i, j] = B[i, obs[j]] * (alpha[:, j - 1] @ A[:, i])

    return alpha

lines = None
with open('q1-test.json') as f:
    lines = f.readlines()
s=''
for i in lines:
    s += i
    
temp = json.loads(s)
observs_test = [None]*len(temp)
for idx, i in enumerate(temp):
    observs_test[idx] = [ord(j) - ord('a') for j in i]
for i in range(len(observs)):
    observs_test[i] = np.array(observs_test[i])
    
for idx, i in enumerate(observs_test):
    print(f'Accuracy {idx} : {forward_temp()}')

TypeError: forward_temp() missing 4 required positional arguments: 'A', 'B', 'pi', and 'obs'